In [1]:
import warnings
warnings.filterwarnings("ignore")
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton, QWidget, QLabel, QGridLayout, QTextEdit, QMessageBox, QComboBox
from PyQt5.QtGui import QFont, QImage, QPixmap
from PyQt5.QtCore import Qt
import numpy as np
import sys
import cv2
import time
import os
import dlib
from imutils import face_utils
import matplotlib.pyplot as plt

import shutil
from keras.utils.np_utils import to_categorical
from keras.applications.resnet_v2 import ResNet50V2
from keras.models import Sequential
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from keras.callbacks import ReduceLROnPlateau
import pandas as pd
import tensorflow as tf
from PIL import Image
from tensorflow import keras

K = tf.keras.backend

In [2]:
nameToAdd = ""
person_id_make_data = 0
list_of_person = []
data_path = "../data/"
brut_path = "../data/brut/"
resize_path = "../data/resize/"


In [3]:
if not os.path.exists(data_path):
    os.makedirs(data_path)

if not os.path.exists(brut_path):
    os.makedirs(brut_path)

if not os.path.exists(resize_path):
    os.makedirs(resize_path)

In [4]:
def addImage(person_ID, person_NAME, img_ID, face_IMG_file_name, dataset): #Ajoute un fichier et toutes ses informations au dataframe. (Insère une ligne)
    new_row = {
        'person_id': person_ID,
        'person_name': person_NAME,
        'img_id': img_ID,
        'face_img_file_name': face_IMG_file_name
        }
    dataset = dataset.append(new_row, ignore_index=True)
    return dataset

In [5]:
df_faces = { #Colonnes du dataframe
    'person_id': [],
    'person_name': [],
    'img_id': [],
    'face_img_file_name': []
    }

In [6]:
def make_data(folder_path, person_id_make_data, name, data): #Crée le dataframe

    dataframe = pd.DataFrame(data)
    person_id_link = []
    
    persons = os.listdir(folder_path)
    
    images_files_names = os.listdir(folder_path)
    
    id_name_tuple = (person_id_make_data, name)
    person_id_link.append(id_name_tuple)
    
    for image_index in range(len(images_files_names)):
        img_file_name = images_files_names[image_index]
        image_id = int(img_file_name.split('_')[-1].split('.')[0])
        face_image_file_name = folder_path + "/" + persons[image_index]
        
        dataframe = addImage(
            person_ID = person_id_make_data,
            person_NAME = name,
            img_ID = image_id,
            face_IMG_file_name = face_image_file_name,
            dataset = dataframe
        )
    person_id_make_data += 1
    dataframe = dataframe.astype({'person_id': 'int32'})
    dataframe = dataframe.astype({'img_id': 'int32'})
    list_of_person.append(name)
    dataframe.to_csv("df_faces.csv", index=False, header=True)
    return dataframe, person_id_make_data, list_of_person

In [7]:
def gray_image(image): #Renvoie l'image d'entrée en niveau de gris
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

In [8]:
def normalize_images(list_of_images): #Normalise les images
    l = len(list_of_images)
    for i in range(l):
        list_of_images[i] = list_of_images[i]/255.
    return list_of_images

In [9]:
def preprocess_data(X, y, gray): #Rend les données utilisables par le modèle. gray ---> booléen, True si on veut les images en niveau de gris, False pour RGB.
    if gray:
        for i in range(len(X)):
            X[i] = gray_image(X[i])
        X = np.asarray(X)
        X = X/255.
        X = X.reshape(X.shape[0], 224, 224, 1)
    else:
        X = normalize_images(X)
        X = np.asarray(X)
        X = X.reshape(X.shape[0], 224, 224, 3)
    y = to_categorical(y)
    return X, y

In [10]:
def get_data(df): #Entrée: dataframe et chemin du dossier data. Renvoie X (les images) et y (les labels).
    IDs = list(df['person_id'])
    FILEs = list(df['face_img_file_name'])
    number_of_images = len(IDs)

    print(IDs)
    print(FILEs)
    print(number_of_images)
    
    X = []
    y = []
    
    for index in range(number_of_images):
        ID = int(IDs[index])
        file_path = FILEs[index]
        image = plt.imread(file_path)
        X.append(image)
        y.append(ID)
    return X, y

In [11]:
def resnet_model_tf(input_shape, nombre_classes):
    resnet = ResNet50V2(weights='imagenet', include_top=False, input_shape=input_shape)
    resnet.tbatch_sizenable = False
    model = Sequential()
    model.add(resnet)
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(nombre_classes, activation='softmax'))
    
    print(model.summary())
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.7, 
                                            min_lr=0.00000000001)
    return model, learning_rate_reduction

In [12]:
def preprocess_input(x, data_format=None, version=1):
    x_temp = np.copy(x)
    if data_format is None:
        data_format = K.image_data_format()
    assert data_format in {'channels_last', 'channels_first'}

    if version == 1:
        if data_format == 'channels_first':
            x_temp = x_temp[:, ::-1, ...]
            x_temp[:, 0, :, :] -= 93.5940
            x_temp[:, 1, :, :] -= 104.7624
            x_temp[:, 2, :, :] -= 129.1863
        else:
            x_temp = x_temp[..., ::-1]
            x_temp[..., 0] -= 93.5940
            x_temp[..., 1] -= 104.7624
            x_temp[..., 2] -= 129.1863

    elif version == 2:
        if data_format == 'channels_first':
            x_temp = x_temp[:, ::-1, ...]
            x_temp[:, 0, :, :] -= 91.4953
            x_temp[:, 1, :, :] -= 103.8827
            x_temp[:, 2, :, :] -= 131.0912
        else:
            x_temp = x_temp[..., ::-1]
            x_temp[..., 0] -= 91.4953
            x_temp[..., 1] -= 103.8827
            x_temp[..., 2] -= 131.0912
    else:
        raise NotImplementedError

    return x_temp

In [13]:
def delete_case(array,indice) : #Supprime la case d'un tableau selon sa position
    newArray = []
    l = len(array)
    for i in range(l) :
        if indice != i :
            newArray = newArray + [array[i]]
    return newArray

In [14]:
def find_minimum(array) : #Renvoie la position de la plus petite valeur dans un tableau
    l = len(array)
    position = 0
    minimum = array[0]
    for i in range(1, l) :
        if array[i] < minimum :
            position = i
            minimum = array[i]
    return position

In [15]:
def sort_arrays(array1, array2) : #Trie array2 selon array1 dans l'ordre croissant
    newArray1 = []
    newArray2 = []
    l = len(array1)
    for i in range(l) :
        posMinimum = find_minimum(array1)
        newArray1 = newArray1 + [array1[posMinimum]]
        newArray2 = newArray2 + [array2[posMinimum]]
        array1 = delete_case(array1, posMinimum)
        array2 = delete_case(array2, posMinimum)
    return newArray1, newArray2

In [16]:
def reorganize(files_list):
    num_list = []
    for fileName in files_list:
        number = int(fileName.split('.')[0].split(' ')[-1])
        num_list.append(number)
    
    num_list, files_list = sort_arrays(num_list, files_list)
    return files_list

In [17]:
def is_global_folder(folder_name):
    if "_FACES" in folder_name:
        return False
    return True


In [18]:
def define_names():
    #names_list = []
    folders = os.listdir(brut_path)
    folders = filter(is_global_folder, folders)
    return folders

In [19]:
#Renvoie le nom d'un fichier sans son extension.
def getRidOfExtenstion(fileName):
    return fileName.split('.')[0]

In [20]:
#Renvoie l'image du visage d'une personne sur l'image entrée en paramètre.
def getCroppedFace(image):
    face_detector = dlib.get_frontal_face_detector()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_detector(gray, 0)
    if len(faces) == 1:
        for face in faces:
            face_bounding_box = face_utils.rect_to_bb(face)
            if all(i >= 0 for i in face_bounding_box):
                [x, y, w, h] = face_bounding_box
                frame = image[y:y + h, x:x + w]
                frame = cv2.resize(frame, (224, 224))
                return frame
    else:
        return 0
    return 0

In [21]:
def treatPictures(name):
    total_pictures = 0
    detected_faces = 0
    pathToFolder = brut_path + name
    images_files = os.listdir(pathToFolder) #Liste de toutes les images.jpg de cette personne.
    number_of_images = len(images_files)
    face_images = []
    face_images_files_names = []
    for image_number in range(number_of_images):
        total_pictures += 1
        image_file_name = str(images_files[image_number])
        image_real_number = image_file_name.split(' ')[-1].split('.')[0] #On recupere le numero de l'image à partir du nom du fichier.
        image_path = pathToFolder + '/' + image_file_name
        image = plt.imread(image_path)
        cropped_face = getCroppedFace(image) #On récupère le visage sur la photo.
        
        if type(cropped_face) != int: #Si un seul visage a été trouvé sur la photo.
            detected_faces += 1
            face_images.append(cropped_face)
            face_image_file_name = name + "_face_" + image_real_number + ".jpg"
            face_images_files_names.append(face_image_file_name)
    
    if len(face_images) != 0: #Si au moins un visage a été détecté sur toutes les photos de la personne.
        path_face_folder = resize_path + name + "_FACES"
        os.mkdir(path_face_folder) #Création du dossier qui va contenir les photos de tous ses visages.
        for face_image_index in range(len(face_images)):
            path_to_write = path_face_folder + "/" + face_images_files_names[face_image_index]
            try:
                cv2.imwrite(path_to_write, cv2.cvtColor(face_images[face_image_index], cv2.COLOR_RGB2BGR)) #Création du fichier .jpg.
            except:
                pass
    message = str(detected_faces) + " faces detected on " + str(total_pictures) + " pictures."
    return message

In [22]:
def increase_brightness(img, value):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

In [23]:
class MainWindow(QMainWindow): #Fenetre principale

    def __init__(self):
        super().__init__()
        self.initUI()
        self.camActivated = False
        self.identification = "Unknown"
        self.explore_index = 0
        self.explore_list = []
        self.explore_filepath = ""
        self.explore_name = ""

    def fit_model(self,df_faces,person_id_make_data):
        name = os.listdir(resize_path)
        if (len(name) != 0):
            for e in name :
                df_faces, person_id_make_data, list_of_person = make_data(resize_path + e, person_id_make_data, e.split("_")[0], df_faces)
            X, y = get_data(df_faces)

            gray_images = False # Mettre True pour avoir les images en niveau de gris, False pour RGB
            X, y = preprocess_data(X, y, gray_images)

            cnn_model, learning_rate_reduction= resnet_model_tf((224, 224, 3), len(os.listdir(resize_path)))
            cnn_model.fit(X, y, epochs=3, verbose=1, callbacks=[learning_rate_reduction])
            cnn_model.save(data_path + "cnn_model")
        else :
            self.popup = QMessageBox(QMessageBox.Information,'Message',"Cannot fit")
            self.popup.show()
    
    def initUI(self):
        self.setWindowTitle('Camera')
        
        self.layout = QGridLayout()
        
        self.screen = QLabel("")
        self.screen.setMinimumHeight(640)
        self.screen.setMinimumWidth(800)
        self.screen.setMaximumHeight(640)
        self.screen.setMaximumWidth(800)
        self.screen.setStyleSheet('background-color: black')
        self.layout.addWidget(self.screen, 0, 0, 1, 2)
        
        self.camButton = QPushButton("Activate Camera")
        self.camButton.setMinimumHeight(70)
        self.camButton.setMinimumWidth(397)
        self.camButton.setMaximumHeight(70)
        self.camButton.setMaximumWidth(397)
        self.camButton.setStyleSheet('background-color: darkgreen')
        self.camButton.setFont(QFont('Arial Black', 15))
        self.camButton.clicked.connect(self.camInteraction)
        self.layout.addWidget(self.camButton, 1, 0)
        
        self.takePicButton = QPushButton("Take Pictures")
        self.takePicButton.setMinimumHeight(70)
        self.takePicButton.setMinimumWidth(397)
        self.takePicButton.setMaximumHeight(70)
        self.takePicButton.setMaximumWidth(397)
        self.takePicButton.setStyleSheet('background-color: forestgreen')
        self.takePicButton.setFont(QFont('Arial Black', 15))
        self.takePicButton.clicked.connect(self.takePictures)
        self.layout.addWidget(self.takePicButton, 1, 1)
        self.takePicButton.setVisible(False)
        
        self.explore = QPushButton("Explore")
        self.explore.setMinimumHeight(70)
        self.explore.setMinimumWidth(397)
        self.explore.setMaximumHeight(70)
        self.explore.setMaximumWidth(397)
        self.explore.setStyleSheet('background-color: grey')
        self.explore.setFont(QFont('Arial Black', 15))
        self.explore.clicked.connect(self.exploring)
        self.layout.addWidget(self.explore, 1, 1)
        
        self.nameZone = QTextEdit("")
        self.nameZone.setMinimumHeight(47)
        self.nameZone.setMinimumWidth(800)
        self.nameZone.setMaximumHeight(47)
        self.nameZone.setMaximumWidth(800)
        self.nameZone.setStyleSheet('background-color: lightgrey')
        self.nameZone.setFont(QFont('Arial', 18))
        self.layout.addWidget(self.nameZone, 2, 0, 1, 2)
        self.nameZone.setVisible(False)
        
        self.explore_layout = QGridLayout() ##################################### EXPLORE
        
        self.buttons_LR_layout = QGridLayout()
        
        self.names = QComboBox()
        self.names.setMinimumHeight(50)
        self.names.setMinimumWidth(380)
        self.names.setMaximumHeight(50)
        self.names.setMaximumWidth(380)
        self.names.setFont(QFont('Arial', 14))
        self.names.setStyleSheet('background-color: lightsteelblue')
        self.explore_layout.addWidget(self.names, 0, 0)
        
        self.search = QPushButton("Search")
        self.search.setMinimumHeight(50)
        self.search.setMinimumWidth(120)
        self.search.setMaximumHeight(50)
        self.search.setMaximumWidth(120)
        self.search.setStyleSheet('background-color: seagreen')
        self.search.setFont(QFont('Arial Black', 15))
        self.search.clicked.connect(self.search_explore)
        self.buttons_LR_layout.addWidget(self.search, 0, 0)
        
        self.left_explore = QPushButton("<")
        self.left_explore.setMinimumHeight(50)
        self.left_explore.setMinimumWidth(120)
        self.left_explore.setMaximumHeight(50)
        self.left_explore.setMaximumWidth(120)
        self.left_explore.setStyleSheet('background-color: royalblue')
        self.left_explore.setFont(QFont('Arial Black', 15))
        self.left_explore.clicked.connect(self.left_exploring)
        self.buttons_LR_layout.addWidget(self.left_explore, 0, 1)
        
        self.right_explore = QPushButton(">")
        self.right_explore.setMinimumHeight(50)
        self.right_explore.setMinimumWidth(120)
        self.right_explore.setMaximumHeight(50)
        self.right_explore.setMaximumWidth(120)
        self.right_explore.setStyleSheet('background-color: royalblue')
        self.right_explore.setFont(QFont('Arial Black', 15))
        self.right_explore.clicked.connect(self.right_exploring)
        self.buttons_LR_layout.addWidget(self.right_explore, 0, 2)
        
        self.buttons_LR_widget = QWidget()
        self.buttons_LR_widget.setLayout(self.buttons_LR_layout)
        self.explore_layout.addWidget(self.buttons_LR_widget, 0, 1)
        
        self.file_name_zone = QLabel("File Name")
        self.file_name_zone.setMinimumHeight(50)
        self.file_name_zone.setMinimumWidth(380)
        self.file_name_zone.setMaximumHeight(50)
        self.file_name_zone.setMaximumWidth(380)
        self.file_name_zone.setStyleSheet('background-color: lightgrey')
        self.file_name_zone.setFont(QFont('Arial Black', 15))
        self.explore_layout.addWidget(self.file_name_zone, 1, 0)
        
        self.quit_explore = QPushButton("Quit")
        self.quit_explore.setMinimumHeight(50)
        self.quit_explore.setMinimumWidth(380)
        self.quit_explore.setMaximumHeight(50)
        self.quit_explore.setMaximumWidth(380)
        self.quit_explore.setStyleSheet('background-color: firebrick')
        self.quit_explore.setFont(QFont('Arial Black', 15))
        self.quit_explore.clicked.connect(self.quit_exploring)
        self.explore_layout.addWidget(self.quit_explore, 1, 1)
        
        self.explore_widget = QWidget()
        self.explore_widget.setLayout(self.explore_layout)
        self.layout.addWidget(self.explore_widget, 3, 0, 1, 2)
        self.explore_widget.setVisible(False)
        
        ######################################################################### END EXPLORE
        
        self.widget = QWidget()
        self.widget.setLayout(self.layout)
        self.setCentralWidget(self.widget)
        
        self.setFixedSize(822, 800)
        self.move(700, 100)
        
    def camInteraction(self):
        if not self.camActivated:
            self.screen.setStyleSheet('background-color: grey')
            self.camButton.setStyleSheet('background-color: darkred')
            self.camButton.setText("Deactivate Camera")
            #self.takePicButton.setStyleSheet('background-color: forestgreen')
            self.explore.setVisible(False)
            self.takePicButton.setVisible(True)
            self.nameZone.setVisible(True)
            self.camActivated = True
            self.cap = cv2.VideoCapture(1) ###
            
            face_detector = dlib.get_frontal_face_detector()

            if os.path.exists(data_path +  "cnn_model"):
                cnn_model = keras.models.load_model(data_path +  "cnn_model")
                check_model = True
            else :
                cnn_model = Sequential()
                check_model = False

            while self.camActivated == True:
                ret, cv_img = self.cap.read()

                ##################################################### PARTIE DETECTION DE VISAGE
                
                gray = cv2.cvtColor(cv_img, cv2.COLOR_BGR2GRAY)

                face_detector = dlib.get_frontal_face_detector()
                faces = face_detector(gray, 0)

                for face in faces:
                    face_bounding_box = face_utils.rect_to_bb(face)
                    if all(i >= 0 for i in face_bounding_box):
                        [x, y, w, h] = face_bounding_box
                        frame = cv_img[y:y + h, x:x + w]
                        cv2.rectangle(cv_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                        frame = cv2.resize(frame, (224, 224))
                        frame = np.asarray(frame, dtype=np.float64)
                        frame = np.expand_dims(frame, axis=0)
                        # frame = preprocess_input(frame)
                        
                        name = "Unknown"
                        if (check_model):
                            prediction = cnn_model.predict(frame)
                            
                            max_value = max(prediction[0])

                            index = prediction[0].tolist().index(max_value)

                            print(max_value)
                            if (max_value > 0.7):
                                name = list_of_person[index]
                            
                        font_face = cv2.FONT_HERSHEY_SIMPLEX
                        cv2.putText(cv_img, name, (x, y-5), font_face, 0.8, (0,0,255), 3)

                ################################################################################
                frame = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
                qimage = QImage(frame, frame.shape[1], frame.shape[0], QImage.Format_RGB888)
                pixmap = QPixmap(qimage)
                pixmap = pixmap.scaled(960,640, Qt.KeepAspectRatio)
                self.screen.setPixmap(pixmap)
                cv2.waitKey(1)
            self.cap.release()
        else:
            im_np = np.zeros((960,540,1))
            qimage = QImage(im_np, im_np.shape[1], im_np.shape[0], QImage.Format_RGB888)
            pixmap = QPixmap(qimage)    
            pixmap = pixmap.scaled(640,400, Qt.KeepAspectRatio)
            self.screen.setPixmap(pixmap)
            self.screen.setStyleSheet('background-color: black')
            self.camButton.setStyleSheet('background-color: darkgreen')
            #self.takePicButton.setStyleSheet('background-color: grey')
            self.takePicButton.setVisible(False)
            self.explore.setVisible(True)
            self.camButton.setText("Activate Camera")
            self.nameZone.setVisible(False)
            self.camActivated = False
    
    def takePictures(self):
        if self.camActivated:
            name = self.nameZone.toPlainText()
            if name == "":
                self.popup = QMessageBox(QMessageBox.Information,'Message','Please enter your name first.')
                self.popup.show()
            else:
                self.nameZone.setText("")
                folders = os.listdir("./")
                if name not in folders:
                    os.mkdir(brut_path + name)
                    count = 0
                    total = 0
                    timeBase = time.time()
                    while total < 20:
                        retCatch, frameCatch = self.cap.read()
                    
                        font = cv2.FONT_HERSHEY_SIMPLEX
                        org = (20, 70)
                        fontScale = 0.8
                        color = (255, 0, 0)
                        thickness = 2
                        percentage = str(total*5) + " %"
                    
                        cv2.waitKey(1)
                    
                        count += 1
                        actualTime = time.time()
                        deltaNeeded = total*0.3
                        deltaActual = actualTime - timeBase
                        if deltaActual >= deltaNeeded:
                            frame = frameCatch
                            if total < 5:
                                frame = increase_brightness(frameCatch, total * 20)
                            if total >=5 and total < 10:
                                frame = cv2.blur(frame, (5, 5)) 
                            cv2.imwrite(os.path.join(brut_path, name + '/' + name + ' ' + str(total) + '.jpg') , frame)
                            total += 1
                        frameCatch = cv2.cvtColor(frameCatch, cv2.COLOR_BGR2RGB)
                        frameOut = cv2.putText(frameCatch, percentage, org, font, fontScale, color, thickness, cv2.LINE_AA)
                        qimage = QImage(frameOut, frameOut.shape[1], frameOut.shape[0], QImage.Format_RGB888)
                        pixmap = QPixmap(qimage)
                        pixmap = pixmap.scaled(960,640, Qt.KeepAspectRatio)
                        self.screen.setPixmap(pixmap)
                        
                    
                    message = treatPictures(name)
                    
                    # self.popup = QMessageBox(QMessageBox.Information,'Message',"Pictures taken!\n" + message)
                    # self.popup.show()
                    self.fit_model(df_faces,person_id_make_data)
                else:
                    self.popup = QMessageBox(QMessageBox.Information,'Message','Please delete the existing folder.')
                    self.popup.show()

        else:
            pass
    
    def exploring(self):
        self.takePicButton.setVisible(False)
        self.camButton.setVisible(False)
        self.explore.setVisible(False)
        self.explore_widget.setVisible(True)
        
        folders = define_names()
        self.names.clear()
        self.names.addItems(folders)
        self.file_name_zone.setText("File Name")
        
    def explore_print(self, filePath): #Affiche la photo à l'ecran selon son path.
        image = plt.imread(filePath)
        qimage = QImage(image, image.shape[1], image.shape[0], QImage.Format_RGB888)
        pixmap = QPixmap(qimage)
        pixmap = pixmap.scaled(960,640, Qt.KeepAspectRatio)
        fileName = filePath.split("/")[-1]
        self.file_name_zone.setText(fileName)
        self.screen.setPixmap(pixmap)
        
    
    def left_exploring(self):
        self.explore_index -= 1
        files_num = len(self.explore_list)
        if self.explore_index == -1:
            self.explore_index = files_num - 1
        self.explore_filepath = brut_path + self.name + "/" + self.explore_list[self.explore_index]
        self.explore_print(self.explore_filepath)
    
    def right_exploring(self):
        self.explore_index += 1
        files_num = len(self.explore_list)
        if self.explore_index == files_num:
            self.explore_index = 0
        self.explore_filepath = brut_path + self.name + "/" + self.explore_list[self.explore_index]
        self.explore_print(self.explore_filepath)
        
    
    def search_explore(self):
        self.name = self.names.currentText()
        self.explore_index = 0
        self.explore_list = os.listdir(brut_path + self.name)
        self.explore_list = reorganize(self.explore_list)
        self.explore_filepath = brut_path + self.name + "/" + self.explore_list[self.explore_index]
        self.explore_print(self.explore_filepath)
        print("search")
    
    def quit_exploring(self):
        self.explore_widget.setVisible(False)
        self.camButton.setVisible(True)
        self.explore.setVisible(True)
        im_np = np.zeros((960,540,1))
        qimage = QImage(im_np, im_np.shape[1], im_np.shape[0], QImage.Format_RGB888)
        pixmap = QPixmap(qimage)
        pixmap = pixmap.scaled(640,400, Qt.KeepAspectRatio)
        self.screen.setPixmap(pixmap)
        self.screen.setStyleSheet('background-color: black')
    
    def closeEvent(self, event): #Fonction qui se lance lors de la fermeture de la fenetre.
        if self.camActivated:
            self.cap.release()
        else:
            pass

In [24]:
app = QApplication(sys.argv)
window = MainWindow()
window.show()
app.exec()

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
['../memory/resize/agnes_FACES/agnes_face_1.jpg', '../memory/resize/agnes_FACES/agnes_face_2.jpg', '../memory/resize/agnes_FACES/agnes_face_3.jpg', '../memory/resize/agnes_FACES/agnes_face_4.jpg', '../memory/resize/agnes_FACES/agnes_face_5.jpg', '../memory/resize/agnes_FACES/agnes_face_6.jpg', '../memory/resize/agnes_FACES/agnes_face_7.jpg', '../memory/resize/agnes_FACES/agnes_face_8.jpg', '../memory/resize/agnes_FACES/agnes_face_9.jpg', '../memory/resize/Jahedul_FACES/Jahedul_face_0.jpg', '../memory/resize/Jahedul_FACES/Jahedul_face_1.jpg', '../memory/resize/Jahedul_FACES/Jahedul_face_10.jpg', '../memory/resize/Jahedul_FACES/Jahedul_face_11.jpg', '../memory/resize/Jahedul_FACES/Jahedul_face_12.jpg', '../memory/resize/Jahedul_FACES/Jahedul_face_13.jpg', '../memory/resize/Jahedul_FA

error: OpenCV(4.5.4) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


0